In [ ]:
!pip install git+https://github.com/zhanghang1989/ResNeSt

In [2]:
from glob import glob
import pandas as pd
from sklearn.model_selection import GroupKFold
import cv2
from skimage import io
import albumentations as A
import torch
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
from tqdm import tqdm
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.nn import functional as F
from glob import glob
import sklearn
from torch import nn
import warnings

warnings.filterwarnings("ignore") 
warnings.filterwarnings("ignore", category=DeprecationWarning) 

SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

# Augmentations

In [3]:
##################################################################################################################
#############################Change Image dims, cutout sizes, cutouts; numbers for experimenting###################
##################################################################################################################
def get_train_transforms():
    return A.Compose([
            A.RandomSizedCrop(min_max_height=(360, 360), height=384, width=384, p=0.5),
            A.RandomRotate90(p=0.5),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.Resize(height=384, width=384, p=1),
            A.Cutout(num_holes=12, max_h_size=24, max_w_size=24, fill_value=0, p=0.5),
            ToTensorV2(p=1.0),                  
        ], p=1.0)

def get_valid_transforms():
    return A.Compose([
            A.Resize(height=384, width=384, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)

# Dataset

In [4]:
FIRST_TRAIN_ROOT_PATH = f'../input/jpeg-melanoma-384x384/train/'
SEC_TRAIN_ROOT_PATH = '../input/jpeg-isic2019-384x384/train/'
THIRD_TRAIN_ROOT_PATH = '../input/malignant-v2-384x384/jpeg384/'

def onehot(size, target):
    vec = torch.zeros(size, dtype=torch.float32)
    vec[target] = 1.
    return vec

class DatasetRetriever(Dataset):

    def __init__(self, image_ids, labels, transforms=None):
        super().__init__()
        self.image_ids = image_ids
        self.labels = labels
        self.transforms = transforms

    def __getitem__(self, idx: int):
        image_id = self.image_ids[idx]
        if 'external_' in image_id:
            image_id = image_id[9:]
            image = cv2.imread(f'{SEC_TRAIN_ROOT_PATH}/{image_id}.jpg', cv2.IMREAD_COLOR)
        elif 'extra_' in image_id:
            image_id = image_id[6:]
            image = cv2.imread(f'{THIRD_TRAIN_ROOT_PATH}/{image_id}.jpg', cv2.IMREAD_COLOR)
        else:
            image = cv2.imread(f'{FIRST_TRAIN_ROOT_PATH}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = image.astype(np.float32) / 255.0

        label = self.labels[idx]

        if self.transforms:
            sample = {'image': image}
            sample = self.transforms(**sample)
            image = sample['image']

        target = onehot(2, label)
        return image, target

    def __len__(self) -> int:
        return self.image_ids.shape[0]

    def get_labels(self):
        return list(self.labels)

In [5]:
train_csv = pd.read_csv('../input/all-files/all_file_512.csv')
skf = StratifiedShuffleSplit(5,random_state=42)

# Metrics

In [6]:
from sklearn import metrics

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


class RocAucMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.y_true = np.array([0,1])
        self.y_pred = np.array([0.5,0.5])
        self.score = 0

    def update(self, y_true, y_pred):
        y_true = y_true.cpu().numpy().argmax(axis=1).clip(min=0, max=1).astype(int)
        # y_pred = 1 - nn.functional.softmax(y_pred, dim=1).data.cpu().numpy()[:,0]
        y_pred = nn.functional.softmax(y_pred, dim=1).data.cpu().numpy()[:,1]
        self.y_true = np.hstack((self.y_true, y_true))
        self.y_pred = np.hstack((self.y_pred, y_pred))
        self.score = sklearn.metrics.roc_auc_score(self.y_true, self.y_pred)

    @property
    def avg(self):
        return self.score

    
class APScoreMeter(RocAucMeter):
    def __init__(self):
        super(APScoreMeter, self).__init__()

    def update(self, y_true, y_pred):
        y_true = y_true.cpu().numpy().argmax(axis=1).clip(min=0, max=1).astype(int)
        y_pred = nn.functional.softmax(y_pred, dim=1).data.cpu().numpy()[:,1]
        self.y_true = np.hstack((self.y_true, y_true))
        self.y_pred = np.hstack((self.y_pred, y_pred))
        self.score = sklearn.metrics.average_precision_score(self.y_true, self.y_pred)

# Loss

In [7]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, logits=False, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduce = reduce

    def forward(self, inputs, targets):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets)
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets)

        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss
        
class LabelSmoothing(nn.Module):
    def __init__(self, smoothing = 0.1):
        super(LabelSmoothing, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing

    def forward(self, x, target):
        if self.training:
            x = x.float()
            target = target.float()
            logprobs = torch.nn.functional.log_softmax(x, dim = -1)

            nll_loss = -logprobs * target
            nll_loss = nll_loss.sum(-1)
    
            smooth_loss = -logprobs.mean(dim=-1)

            loss = self.confidence * nll_loss + self.smoothing * smooth_loss

            return loss.mean()
        else:
            return torch.nn.functional.cross_entropy(x, target)

# Net

In [8]:
def get_net():
    net = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50_fast_2s1x64d', pretrained=True)
    ## resnest50_fast_2s1x64d can be chaged to different models.
    ## refer to https://github.com/zhanghang1989/ResNeSt
    net.fc = nn.Linear(in_features=2048, out_features=2, bias=True)
    return net
net = get_net().cuda()

Downloading: "https://github.com/zhanghang1989/ResNeSt/archive/master.zip" to /root/.cache/torch/hub/master.zip
Downloading: "https://s3.us-west-1.wasabisys.com/resnest/torch/resnest50_fast_2s1x64d-44938639.pth" to /root/.cache/torch/hub/checkpoints/resnest50_fast_2s1x64d-44938639.pth


# Fitter

In [9]:
class Fitter:
    
    def __init__(self, model, device, config, folder):
        self.config = config
        self.epoch = 0

        self.base_dir = f'./{folder}'
        if not os.path.exists(self.base_dir):
            os.makedirs(self.base_dir)

        self.log_path = f'{self.base_dir}/log.txt'
        self.best_score = 0
        self.best_loss = 10**5
        self.best_ap = 0
        
        self.model = model
        self.device = device

        param_optimizer = list(self.model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ] 

        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=config.lr)
        self.scheduler = config.SchedulerClass(self.optimizer, **config.scheduler_params)

#         self.criterion = FocalLoss(logits=True).to(self.device)
        self.criterion = LabelSmoothing().to(self.device)
        self.log(f'Fitter prepared. Device is {self.device}')

    def fit(self, train_loader, validation_loader):
        for e in range(self.config.n_epochs):
            if self.config.verbose:
                lr = self.optimizer.param_groups[0]['lr']
                timestamp = datetime.utcnow().isoformat()
                self.log(f'\n{timestamp}\nLR: {lr}')

            t = time.time()
            summary_loss, roc_auc_scores, ap_scores = self.train_one_epoch(train_loader)
            self.log(f'[RESULT]: Train. Epoch: {self.epoch}, summary_loss: {summary_loss.avg:.5f}, roc_auc: {roc_auc_scores.avg:.5f}, ap: {ap_scores.avg:.5f}, time: {(time.time() - t):.5f}')

            t = time.time()
            summary_loss, roc_auc_scores, ap_scores = self.validation(validation_loader)

            self.log(f'[RESULT]: Val. Epoch: {self.epoch}, summary_loss: {summary_loss.avg:.5f}, roc_auc: {roc_auc_scores.avg:.5f}, ap: {ap_scores.avg:.5f}, time: {(time.time() - t):.5f}')
            if summary_loss.avg < self.best_loss:
                self.best_loss = summary_loss.avg
                self.save_model(f'{self.base_dir}/best-loss-checkpoint-{str(self.epoch).zfill(3)}epoch.bin')
                for path in sorted(glob(f'{self.base_dir}/best-loss-checkpoint-*epoch.bin'))[:-2]:
                    os.remove(path)
                    
            if roc_auc_scores.avg > self.best_score:
                self.best_score = roc_auc_scores.avg
                self.save_model(f'{self.base_dir}/best-score-checkpoint-{str(self.epoch).zfill(3)}epoch.bin')
                for path in sorted(glob(f'{self.base_dir}/best-score-checkpoint-*epoch.bin'))[:-2]:
                    os.remove(path)
                    
            if ap_scores.avg > self.best_ap:
                self.best_ap = ap_scores.avg
                self.save_model(f'{self.base_dir}/best-ap-checkpoint-{str(self.epoch).zfill(3)}epoch.bin')
                for path in sorted(glob(f'{self.base_dir}/best-ap-checkpoint-*epoch.bin'))[:-2]:
                    os.remove(path)

            if self.config.validation_scheduler:
                self.scheduler.step(metrics=roc_auc_scores.avg)

            self.epoch += 1

    def validation(self, val_loader):
        self.model.eval()
        summary_loss = AverageMeter()
        roc_auc_scores = RocAucMeter()
        ap_scores = APScoreMeter()
        t = time.time()
        for step, (images, targets) in enumerate(val_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Val Step {step}/{len(val_loader)}, ' + \
                        f'summary_loss: {summary_loss.avg:.5f}, roc_auc: {roc_auc_scores.avg:.5f}, ap: {ap_scores.avg:.5f} ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            with torch.no_grad():
                targets = targets.to(self.device).float()
                batch_size = images.shape[0]
                images = images.to(self.device).float()
                outputs = self.model(images)
                loss = self.criterion(outputs, targets)
                roc_auc_scores.update(targets, outputs)
                ap_scores.update(targets, outputs)
                summary_loss.update(loss.detach().item(), batch_size)

        return summary_loss, roc_auc_scores, ap_scores

    def train_one_epoch(self, train_loader):
        self.model.train()
        summary_loss = AverageMeter()
        roc_auc_scores = RocAucMeter()
        ap_scores = APScoreMeter()
        t = time.time()
        for step, (images, targets) in enumerate(train_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Train Step {step}/{len(train_loader)}, ' + \
                        f'summary_loss: {summary_loss.avg:.5f}, roc_auc: {roc_auc_scores.avg:.5f}, ap: {ap_scores.avg:.5f} ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            
            targets = targets.to(self.device).float()
            images = images.to(self.device).float()
            batch_size = images.shape[0]

            self.optimizer.zero_grad()
            outputs = self.model(images)
            loss = self.criterion(outputs, targets)
            loss.backward()
            
            roc_auc_scores.update(targets, outputs)
            ap_scores.update(targets, outputs)
            summary_loss.update(loss.detach().item(), batch_size)

            self.optimizer.step()

            if self.config.step_scheduler:
                self.scheduler.step()

        return summary_loss, roc_auc_scores, ap_scores
    
    def save_model(self, path):
        self.model.eval()
        torch.save(self.model.state_dict(),path)

    def save(self, path):
        self.model.eval()
        torch.save({
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'best_score': self.best_score,
            'best_ap': self.best_ap,
            'best_loss': self.best_loss,
            'epoch': self.epoch,
        }, path)

    def load(self, path):
        checkpoint = torch.load(path)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        self.best_score = checkpoint['best_score']
        self.best_ap = checkpoint['best_ap']
        self.best_loss = checkpoint['best_loss']
        self.epoch = checkpoint['epoch']
        
    def log(self, message):
        if self.config.verbose:
            print(message)
        with open(self.log_path, 'a+') as logger:
            logger.write(f'{message}\n')

In [10]:
class TrainGlobalConfig:
    num_workers = 2
    batch_size = 32
    n_epochs = 25
    lr = 7e-5

    # -------------------
    verbose = True
    verbose_step = 1
    # -------------------

    # --------------------
    step_scheduler = False  # do scheduler.step after optimizer.step
    validation_scheduler = True  # do scheduler.step after validation stage loss

#     SchedulerClass = torch.optim.lr_scheduler.OneCycleLR
#     scheduler_params = dict(
#         max_lr=0.001,
#         epochs=n_epochs,
#         steps_per_epoch=int(len(train_dataset) / batch_size),
#         pct_start=0.1,
#         anneal_strategy='cos', 
#         final_div_factor=10**5
#     )
    
    SchedulerClass = torch.optim.lr_scheduler.ReduceLROnPlateau
    scheduler_params = dict(
        mode='max',
        factor=0.8,
        patience=1,
        verbose=False, 
        threshold=0.0001,
        threshold_mode='abs',
        cooldown=0, 
        min_lr=1e-8,
        eps=1e-08
    )
    # --------------------

# Save all states for "honest" training of folds

In [11]:
fitter = Fitter(model=net, device=torch.device('cuda:0'), config=TrainGlobalConfig, folder='base_state')
BASE_STATE_PATH = f'{fitter.base_dir}/base_state.bin'
fitter.save(BASE_STATE_PATH)

Fitter prepared. Device is cuda:0


In [12]:
from catalyst.data.sampler import BalanceClassSampler

def train_fold(fold_number):
    for i,(t_ind,v_ind) in enumerate(skf.split(train_csv,train_csv['target'])):
        if i==fold_number:
            df_train = train_csv.iloc[t_ind]
            df_val = train_csv.iloc[v_ind]
            df_val = df_val[df_val.source=='ISIC_2020']
            
            
    train_dataset = DatasetRetriever(
        image_ids=df_train.image_name.values,
        labels=df_train.target.values,
        transforms=get_train_transforms(),
    )
    validation_dataset = DatasetRetriever(
        image_ids=df_val.image_name.values,
        labels=df_val.target.values,
        transforms=get_valid_transforms(),
    )
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        sampler=BalanceClassSampler(labels=train_dataset.get_labels(), mode="downsampling"),
        batch_size=TrainGlobalConfig.batch_size,
        pin_memory=False,
        drop_last=True,
        num_workers=TrainGlobalConfig.num_workers,
    )
    print(len(train_loader))
    val_loader = torch.utils.data.DataLoader(
        validation_dataset, 
        batch_size=TrainGlobalConfig.batch_size*2,
        num_workers=TrainGlobalConfig.num_workers,
        shuffle=False,
        sampler=SequentialSampler(validation_dataset),
        pin_memory=False,
    )
    print(len(val_loader))
    fitter = Fitter(model=net, device=torch.device('cuda:0'), config=TrainGlobalConfig, folder=f'fold{fold_number}')
    fitter.load(BASE_STATE_PATH)
    fitter.fit(train_loader, val_loader)
    
def test(fold_number:int , tta:int = 25):
    '''
    Runs inference on the test dataset
            Parameters:
                    fold_number : Sr. No. of trained fold
                    tta         :No. of test-time Augmentaions
    '''

    files = glob(f'fold{fold_number}/best-score-checkpoint-*epoch.bin')
    files.sort()
    checkpoint_path = files[-1]
    print(checkpoint_path)
    TEST_ROOT_PATH = f'../input/jpeg-melanoma-512x512/test/'
    class DatasetRetriever(Dataset):

        def __init__(self, image_ids, transforms=None):
            super().__init__()
            self.image_ids = image_ids
            self.transforms = transforms

        def __getitem__(self, idx: int):
            image_id = self.image_ids[idx]
            image = cv2.imread(f'{TEST_ROOT_PATH}/{image_id}.jpg', cv2.IMREAD_COLOR)
            image = image.astype(np.float32) / 255.0
            if self.transforms:
                sample = {'image': image}
                sample = self.transforms(**sample)
                image = sample['image']
            return image, image_id

        def __len__(self) -> int:
            return self.image_ids.shape[0]

    df_test = pd.read_csv(f'../input/siim-isic-melanoma-classification/test.csv', index_col='image_name')

    test_dataset = DatasetRetriever(
        image_ids=df_test.index.values,
        transforms=get_train_transforms(),
    )

    test_loader = torch.utils.data.DataLoader(
        test_dataset, 
        batch_size=TrainGlobalConfig.batch_size*2,
        num_workers=2,
        shuffle=False,
        sampler=SequentialSampler(test_dataset),
        pin_memory=False,
        drop_last=False,
    )
    checkpoint = torch.load(checkpoint_path)
    net.load_state_dict(checkpoint);

    result = {'image_name': [], 'target': []}
    for i in range(tta):
        for images, image_names in tqdm(test_loader, total=len(test_loader)):
            with torch.no_grad():
                images = images.cuda().float()
                outputs = net(images)
                y_pred = nn.functional.softmax(outputs, dim=1).data.cpu().numpy()[:,1]

            result['image_name'].extend(image_names)
            result['target'].extend(y_pred)

    test_res_all = pd.DataFrame(result)
    test_res_all.to_csv(f"test_res_all{fold_number}.csv", index=False)
    test_res_all[['image_name', 'target']].groupby('image_name').mean().reset_index().to_csv(f"tta{tta}_fold{fold_number}_submission.csv", index=False)

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [13]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
for i in range(2, 5):
    torch.cuda.empty_cache()
    train_fold(fold_number=i)
    test(fold_number=i, tta = 25)

158
52
Fitter prepared. Device is cuda:0

2020-08-15T11:42:25.381499
LR: 7e-05
[RESULT]: Train. Epoch: 0, summary_loss: 0.49711, roc_auc: 0.87642, ap: 0.86034, time: 105.92417
[RESULT]: Val. Epoch: 0, summary_loss: 0.36986, roc_auc: 0.84516, ap: 0.08634, time: 26.19828

2020-08-15T11:44:37.893674
LR: 7e-05
[RESULT]: Train. Epoch: 1, summary_loss: 0.44949, roc_auc: 0.91281, ap: 0.90082, time: 100.88515
[RESULT]: Val. Epoch: 1, summary_loss: 0.32343, roc_auc: 0.84938, ap: 0.08219, time: 21.79748

2020-08-15T11:46:40.836755
LR: 7e-05
[RESULT]: Train. Epoch: 2, summary_loss: 0.42591, roc_auc: 0.92739, ap: 0.91817, time: 100.34548
[RESULT]: Val. Epoch: 2, summary_loss: 0.42590, roc_auc: 0.89224, ap: 0.13916, time: 21.86450

2020-08-15T11:48:43.309901
LR: 7e-05
[RESULT]: Train. Epoch: 3, summary_loss: 0.41914, roc_auc: 0.93111, ap: 0.92189, time: 100.76316
[RESULT]: Val. Epoch: 3, summary_loss: 0.35717, roc_auc: 0.89701, ap: 0.15764, time: 22.36528

2020-08-15T11:50:46.724769
LR: 7e-05
[RESU

100%|██████████| 172/172 [02:08<00:00,  1.34it/s]


158
51
Fitter prepared. Device is cuda:0

2020-08-15T13:26:12.669391
LR: 7e-05
[RESULT]: Train. Epoch: 0, summary_loss: 0.50304, roc_auc: 0.87233, ap: 0.85872, time: 100.76476
[RESULT]: Val. Epoch: 0, summary_loss: 0.32396, roc_auc: 0.88536, ap: 0.15426, time: 24.91086

2020-08-15T13:28:18.725289
LR: 7e-05
[RESULT]: Train. Epoch: 1, summary_loss: 0.44901, roc_auc: 0.91298, ap: 0.90297, time: 100.18222
[RESULT]: Val. Epoch: 1, summary_loss: 0.39841, roc_auc: 0.89299, ap: 0.19603, time: 22.14278

2020-08-15T13:30:21.313844
LR: 7e-05
[RESULT]: Train. Epoch: 2, summary_loss: 0.42632, roc_auc: 0.92633, ap: 0.91441, time: 100.18241
[RESULT]: Val. Epoch: 2, summary_loss: 0.43018, roc_auc: 0.90567, ap: 0.25878, time: 22.01147

2020-08-15T13:32:23.788644
LR: 7e-05
[RESULT]: Train. Epoch: 3, summary_loss: 0.40440, roc_auc: 0.93980, ap: 0.93210, time: 100.60826
[RESULT]: Val. Epoch: 3, summary_loss: 0.33553, roc_auc: 0.89389, ap: 0.17443, time: 22.61687

2020-08-15T13:34:27.016527
LR: 7e-05
[RESU

100%|██████████| 172/172 [02:13<00:00,  1.28it/s]


158
52
Fitter prepared. Device is cuda:0

2020-08-15T15:13:23.790028
LR: 7e-05
[RESULT]: Train. Epoch: 0, summary_loss: 0.49849, roc_auc: 0.87541, ap: 0.86357, time: 101.67649
[RESULT]: Val. Epoch: 0, summary_loss: 0.36946, roc_auc: 0.84984, ap: 0.13636, time: 25.36440

2020-08-15T15:15:31.230701
LR: 7e-05
[RESULT]: Train. Epoch: 1, summary_loss: 0.43482, roc_auc: 0.92205, ap: 0.91301, time: 101.12801
[RESULT]: Val. Epoch: 1, summary_loss: 0.31749, roc_auc: 0.88301, ap: 0.20687, time: 23.02249

2020-08-15T15:17:35.768743
LR: 7e-05
[RESULT]: Train. Epoch: 2, summary_loss: 0.43124, roc_auc: 0.92324, ap: 0.91185, time: 101.11626
[RESULT]: Val. Epoch: 2, summary_loss: 0.33425, roc_auc: 0.85667, ap: 0.13204, time: 22.84595

2020-08-15T15:19:39.733745
LR: 7e-05
[RESULT]: Train. Epoch: 3, summary_loss: 0.40754, roc_auc: 0.93811, ap: 0.92790, time: 101.21732
[RESULT]: Val. Epoch: 3, summary_loss: 0.35484, roc_auc: 0.85612, ap: 0.14034, time: 22.34889

2020-08-15T15:21:43.302746
LR: 5.6e-05
[RE

  0%|          | 0/172 [00:00<?, ?it/s]

[RESULT]: Val. Epoch: 24, summary_loss: 0.26002, roc_auc: 0.80817, ap: 0.24516, time: 23.91953
fold4/best-score-checkpoint-005epoch.bin


100%|██████████| 172/172 [02:16<00:00,  1.26it/s]
